# 1. Modelo Conceitual
![Modelo Conceitual](https://github.com/guilherme1905/Projeto-MC536/raw/master/documentos/modelo_conceitual_v3.png)

# 2.Modelo Lógico
![Modelo Lógico](https://github.com/guilherme1905/Projeto-MC536/raw/master/documentos/modelo_logico_v1.png)

In [1]:
%defaultDatasource jdbc:h2:mem:db

## Estados

In [2]:
DROP TABLE IF EXISTS UF;

CREATE TABLE UF (
    nome VARCHAR (20) NOT NULL,
    regiao VARCHAR(5) NOT NULL,
    n_municipios INT NOT NULL,
    PRIMARY KEY (nome),
);

INSERT INTO UF VALUES ('Rondonia', 'N', 52);
INSERT INTO UF VALUES ('Acre', 'N', 22);
INSERT INTO UF VALUES ('Amazonas', 'N', 62);
INSERT INTO UF VALUES ('Roraima', 'N', 15);
INSERT INTO UF VALUES ('Para', 'N', 144);
INSERT INTO UF VALUES ('Amapa', 'N', 16);
INSERT INTO UF VALUES ('Tocantins', 'N', 139);
INSERT INTO UF VALUES ('Minas Gerais', 'SE', 853);
INSERT INTO UF VALUES ('Espirito Santo', 'SE', 78);
INSERT INTO UF VALUES ('Rio de Janeiro', 'SE', 92);
INSERT INTO UF VALUES ('Sao Paulo', 'SE', 645);
INSERT INTO UF VALUES ('Parana', 'S', 399);
INSERT INTO UF VALUES ('Santa Catarina', 'S', 295);
INSERT INTO UF VALUES ('Rio Grande do Sul', 'S', 497);
INSERT INTO UF VALUES ('Mato Grosso do Sul', 'CO', 79);
INSERT INTO UF VALUES ('Mato Grosso', 'CO', 141);
INSERT INTO UF VALUES ('Goias', 'CO', 246);
INSERT INTO UF VALUES ('Distrito Federal', 'CO', 1);
INSERT INTO UF VALUES ('Maranhao', 'NE', 217);
INSERT INTO UF VALUES ('Piaui', 'NE', 224);
INSERT INTO UF VALUES ('Ceara', 'NE', 184);
INSERT INTO UF VALUES ('Rio Grande do Norte', 'NE', 167);
INSERT INTO UF VALUES ('Paraiba', 'NE', 223);
INSERT INTO UF VALUES ('Pernambuco', 'NE', 185);
INSERT INTO UF VALUES ('Alagoas', 'NE', 102);
INSERT INTO UF VALUES ('Sergipe', 'NE', 75);
INSERT INTO UF VALUES ('Bahia', 'NE', 417);

## Regiões do Brasil

In [3]:
DROP TABLE IF EXISTS Regiao;

CREATE TABLE Regiao (
    nome VARCHAR(20) NOT NULL,
    nMunicipios INT NOT NULL,
    PRIMARY KEY(nome)
) AS SELECT
    regiao,
    SUM(n_municipios)
    FROM UF
    GROUP BY regiao;

## Doenças

In [24]:
DROP TABLE IF EXISTS Doenca;

CREATE TABLE Doenca (
    nome VARCHAR(50) NOT NULL,
    contaminacao VARCHAR(50),
    PRIMARY KEY(nome)
) AS SELECT
    Doencas,
    Vetor
    FROM CSVREAD('data/doenca.csv');

## Relação Doença - Estado

In [26]:
DROP TABLE IF EXISTS DoencasPorEstado;

CREATE TABLE DoencasPorEstado (
    estado VARCHAR(20) NOT NULL,
    nomeDoenca VARCHAR(50) NOT NULL,
    nCasos INT NOT NULL,
    FOREIGN KEY(nomeDoenca)
        REFERENCES Doenca(nome)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY(estado)
        REFERENCES UF(nome)
        ON UPDATE CASCADE
        ON DELETE NO ACTION
) AS SELECT
    UF,
    Doenca,
    nCasos
    FROM CSVREAD('data/doencas_estado_saneamento.csv');

## Relação Doença - Região

In [29]:
DROP TABLE IF EXISTS DoencasPorRegiao;

CREATE TABLE DoencasPorRegiao (
    nomeDoenca VARCHAR(50) NOT NULL,
    regiao VARCHAR(2) NULL,
    nCasos INT NOT NULL,
    FOREIGN KEY(nomeDoenca)
        REFERENCES Doenca
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY(regiao)
        REFERENCES Regiao
        ON UPDATE CASCADE
        ON DELETE NO ACTION
) AS SELECT
    DE.nomeDoenca,
    UF.regiao,
    SUM(DE.nCasos)
    FROM DoencasPorEstado DE, UF
    WHERE DE.estado = UF.nome
    GROUP BY regiao, nomeDoenca;

## Tipo de Órgão responsável pela regulação dos serviços de saneamento básico por número de cidades em cada UF

In [42]:
DROP TABLE IF EXISTS PorcOrgaoAgua;
DROP TABLE IF EXISTS PorcOrgaoEsg;
DROP TABLE IF EXISTS OrgaoSaneamentoAgua;
DROP TABLE IF EXISTS OrgaoSaneamentoEsgoto;

CREATE TABLE OrgaoSaneamentoAgua(
    estado VARCHAR(20) NOT NULL,
    OrgaoMunicipal INT,
    AutarquiaMunicipal INT,
    ConsorcioPub INT,
    EntidadeEstadual INT,
    outro_orgao_publico INT,
    outros INT,
    FOREIGN KEY(estado)
        REFERENCES UF
        ON UPDATE CASCADE
        ON DELETE NO ACTION
) AS SELECT
    UF,
    n_orgao_municipal,
    n_autarquia_municipal,
    n_consorcio_pub,
    n_ent_reg_estadual,
    n_outro_orgao_pub,
    n_outros
    FROM CSVREAD('data/orgao_agua.csv');
    
CREATE TABLE OrgaoSaneamentoEsgoto(
    estado VARCHAR(20) NOT NULL,
    OrgaoMunicipal INT,
    AutarquiaMunicipal INT,
    ConsorcioPub INT,
    EntidadeEstadual INT,
    outro_orgao_publico INT,
    outros INT,
    FOREIGN KEY(estado)
        REFERENCES UF
        ON UPDATE CASCADE
        ON DELETE NO ACTION
) AS SELECT
    UF,
    n_orgao_municipal,
    n_autarquia_municipal,
    n_consorcio_pub,
    n_ent_reg_estadual,
    n_outro_orgao_pub,
    n_outros
    FROM CSVREAD('data/orgao_esgoto.csv');

## Queries:

* Casos de doenças que tem transmissão afetadas por condições de saneamento no Brasil (2017)

In [34]:
SELECT nomeDoenca, SUM(nCasos) AS Casos FROM DoencasPorRegiao GROUP BY nomeDoenca;

* Porcentagem de cada tipo de órgão responsável pela regulação dos serviços de saneamento básico por UF

In [52]:
DROP TABLE IF EXISTS PorcOrgaoAguaD;
DROP TABLE IF EXISTS PorcOrgaoEsgD;
DROP TABLE IF EXISTS PorcOrgaoAgua;
DROP TABLE IF EXISTS PorcOrgaoEsg;

CREATE VIEW PorcOrgaoAgua AS
SELECT OSA.estado, 100 * OSA.OrgaoMunicipal / UF.n_municipios OM, 100 * OSA.AutarquiaMunicipal / UF.n_municipios AM, 
       100 * OSA.ConsorcioPub / UF.n_municipios CP, 100 * OSA.EntidadeEstadual / UF.n_municipios EE, 100 * OSA.outro_orgao_publico / UF.n_municipios OOP, 100 * OSA.outros / UF.n_municipios OTHERS
FROM OrgaoSaneamentoAgua OSA, UF
WHERE OSA.estado = UF.nome;

CREATE VIEW PorcOrgaoEsg AS
SELECT OSE.estado, 100 * OSE.OrgaoMunicipal / UF.n_municipios OM, 100 * OSE.AutarquiaMunicipal / UF.n_municipios AM, 
       100 * OSE.ConsorcioPub / UF.n_municipios CP, 100 * OSE.EntidadeEstadual / UF.n_municipios EE, 100 * OSE.outro_orgao_publico / UF.n_municipios OOP, 100 * OSE.outros / UF.n_municipios OTHERS
FROM OrgaoSaneamentoEsgoto OSE, UF
WHERE OSE.estado = UF.nome;

SELECT * FROM PorcOrgaoAgua;
SELECT * FROM PorcOrgaoEsg;

* Porcentagem de cada tipo de órgão responsável pela regulação dos serviços de sanemento básico por UF, com número de casos de doenças

In [49]:
DROP TABLE IF EXISTS PorcOrgaoAguaD;
DROP TABLE IF EXISTS PorcOrgaoEsgD;

CREATE VIEW PorcOrgaoAguaD AS
SELECT POA.estado, DpE.nomeDoenca, DpE.nCasos, OM, AM, CP, EE, OOP, OTHERS
FROM PorcOrgaoAgua POA, DoencasPorEstado DpE
WHERE POA.estado = DpE.estado
ORDER BY nCasos;

CREATE VIEW PorcOrgaoEsgD AS
SELECT POE.estado, DpE.nomeDoenca, DpE.nCasos, OM, AM, CP, EE, OOP, OTHERS
FROM PorcOrgaoEsg POE, DoencasPorEstado DpE
WHERE POE.estado = DpE.estado;

SELECT * FROM PorcOrgaoAguaD;

SELECT * FROM PorcOrgaoEsgD;